Installing Pipekit Python SDK:

In [ ]:
pip install pipekit-sdk

Creating a PipekitService object (token can be obtained by running `pipekit login` and then `pipekit hera`):

In [ ]:
from pipekit_sdk.service import PipekitService

In [ ]:
pipekit = PipekitService(token='<your_token>')

Listing clusters and pipes:

In [ ]:
pipekit.list_clusters()

In [ ]:
pipekit.list_pipes()

Creating a Hera workflow ([Conflip example](https://hera-workflows.readthedocs.io/en/latest/examples/workflows/coinflip/)):

In [ ]:
from hera.workflows import DAG, Workflow, script


@script()
def flip():
    import random

    result = "heads" if random.randint(0, 1) == 0 else "tails"
    print(result)


@script()
def heads():
    print("it was heads")


@script()
def tails():
    print("it was tails")


with Workflow(generate_name="coinflip-", entrypoint="d", namespace="ci", service_account_name="ci") as w:
    with DAG(name="d") as s:
        f = flip()
        heads().on_other_result(f, "heads")
        tails().on_other_result(f, "tails")


Submitting the workflow ("runner" is the name of the cluster):

In [ ]:
pipe_run = pipekit.submit(w, "runner")

Getting the run status and other properties:

In [ ]:
pipekit.get_run(pipe_run["uuid"])

Tailing the logs (add follow=False if you want to get the entire log up to this point):

In [ ]:
pipekit.print_logs(pipe_run["uuid"], container_name="main")

Stopping the run (restart and terminate are also available):

In [ ]:
pipekit.stop(pipe_run["uuid"])